In [1]:
import re
import torch
import pickle
import unicodedata
import pandas as pd
import matplotlib.pyplot as plt

from scipy.special import softmax
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from simpletransformers.classification import ClassificationModel

In [2]:
data = pd.read_csv("../datasets/augmented/9834838408490912248/cards_augmented_0_V1.csv")

In [3]:
with open('../cards/models/label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

/opt/conda/envs/uni/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.0 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
# Load and pre-process the text data
# Define text pre-processing functions
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
def strip_underscores(text):
    return re.sub(r'_+', ' ', text)
def remove_multiple_spaces(text):
    return re.sub(r'\s{2,}', ' ', text)

# Merge text pre-processing functions
def denoise_text(text):
    text = remove_between_square_brackets(text)
    text = remove_non_ascii(text)
    text = strip_underscores(text)
    text = remove_multiple_spaces(text)
    return text.strip()

data["p_text"] = data.text.astype(str).apply(denoise_text)

In [5]:
# Define the model 
architecture = 'roberta'
# model_name = 'CARDS_RoBERTa_Classifier'
model_name = "../cards/models/CARDS_RoBERTa_Classifier"

# Load the classifier
roberta_model = ClassificationModel(architecture, model_name, use_cuda=False)
roberta_model.config.output_hidden_states = True

Some weights of the model checkpoint at ../cards/models/CARDS_RoBERTa_Classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def cluster_data(texts, labels):
    
    preds, model_outputs, all_embedding_outputs, all_layer_hidden_states = roberta_model.predict(texts)
    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    low_dim_embs = tsne.fit_transform(
        all_embedding_outputs.reshape(all_embedding_outputs.shape[0], -1))
    plt.scatter(x=low_dim_embs[:,0], y=low_dim_embs[:,1], c=labels)

data_train = data[data.PARTITION=="TRAIN"]
texts = data.p_text.tolist()
labels = le.transform(data.claim.to_list())
cluster_data(texts, labels)

  0%|          | 0/28945 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1662 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1072 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (856 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (671 > 512). Running this sequence through the model will result in indexing errors


Running Prediction:   0%|          | 0/3633 [00:00<?, ?it/s]

In [ ]:
data_train = data[data.PARTITION=="VALID"]
texts = data.p_text.tolist()
labels = le.transform(data.claim.to_list())
cluster_data(texts, labels)

In [ ]:
data_train = data[data.PARTITION=="TEST"]
texts = data.p_text.tolist()
labels = le.transform(data.claim.to_list())
cluster_data(texts, labels)